In [2]:
!pip install polars
!pip install sentence-transformers
!pip install scikit-learn

In [1]:
import polars as pl
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np



/home/tnsantos/datathon-recrutamento-ia/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

resultado_final_df_polars = pl.read_parquet("/home/tnsantos/datathon-recrutamento-ia/teste4/resultado_final.parquet")
resultado_final_df_polars.head(10).to_pandas()


,codigo_vaga,titulo,modalidade,codigo_candidato,p_situacao_candidado,p_data_candidatura,p_ultima_atualizacao,p_comentario,p_recrutador,app_ib_objetivo_profissional,...,job_area_formacao,app_principais_ferramentas_tecnologicas,app_ferramentas_tecnologicas,app_competencias_tecnicas,app_competencias_comportamentais,app_experiencia_anos,app_senioridade_aparente,app_formacao_academica,app_nivel_formacao,app_area_formacao
0,4530,CONSULTOR CONTROL M,,25632,ENCAMINHADO AO REQUISITANTE,2021-03-25,2021-03-25,"ENCAMINHADO PARA - PJ R 72,00/HORA",ANA LÍVIA MOREIRA,,...,TI,"[PL/SQL, ITIL, Java, DB2, Python, SQL, HTML, D...","[Control-M, PowerCenter, IBM Cloud, Docker, Ku...","[PL/SQL, ITIL, SCM, DB2, Java, Python, SQL, Da...","[Comunicação, Trabalho em equipe, Liderança, R...",8-10 anos,Sênior,Superior Completo,Superior,Tecnologia da Informação
1,4530,CONSULTOR CONTROL M,,25529,ENCAMINHADO AO REQUISITANTE,2021-03-22,2021-03-23,"ENCAMINHADO PARA - R 6.000,00 CLT FULL , NÃO ...",ANA LÍVIA MOREIRA,ANALISTA DE REDES E TELEPROCESSAMENTO/INFRAEST...,...,TI,"[Redes de computadores, Virtualização VMware, ...","[Redes LAN/WAN, Virtualização VMware, Ferramen...","[Administração de Data Centers, Gerenciamento ...","[Comunicação, Trabalho em Equipe, Liderança, A...",8,Pieno,Superior Completo,Superior Completo,TI
2,4531,2021-2607395-PEOPLESOFT APPLICATION ENGINE-DOM...,,25364,CONTRATADO PELA DECISION,2021-03-17,2021-04-12,DATA DE INICIO: 12/04/2021,JULIANA CASSIANO,LIDERANÇA / DESENVOLVIMENTO,...,TI,"[SAP, Java, Python, SQL, Oracle, Power BI, Clo...","[SAP ERP, Java, SQL, Oracle, Power BI, Cloud P...","[SAP Implementation, Java Development, Cloud A...","[Problem Solving, Teamwork, Communication, Lea...",8-10 anos,Sênior,Superior Completo,Superior Completo,TI
3,4531,2021-2607395-PEOPLESOFT APPLICATION ENGINE-DOM...,,25360,ENCAMINHADO AO REQUISITANTE,2021-03-17,2021-03-17,,JULIANA CASSIANO,,...,TI,"[Oracle PeopleSoft, SAP, ABAP, SQL, Relatórios...","[Oracle PeopleTools, SAP, ABAP, SQL, Crystal, ...","[Desenvolvimento, Implementação, Configuração,...","[Resolução de problemas, Comunicação, Trabalho...",8-10 anos,Sênior,true,Superior Completo,TI
4,4533,2021-2605708-MICROFOCUS APPLICATION LIFE CYCLE...,,26338,CONTRATADO PELA DECISION,2021-04-29,2021-05-18,,STELLA VIEIRA,,...,TI,"[Testes, Automação, Integração, Performance, B...","[Java, C#, SQL, Oracle, MySQL, SharePoint, SAP...","[Testes, Automação, Integração, Performance, B...","[Proatividade, Liderança, Adaptabilidade, Comu...",5-8 anos,Pleno,true,Superior Completo,TI
5,4533,2021-2605708-MICROFOCUS APPLICATION LIFE CYCLE...,,24645,DESISTIU,2021-04-27,2021-04-27,PROFISSIONAL DESISTIU DA VAGA. MOTIVO : NA VER...,YASMIN DA ROSA,,...,TI,"[Selenium, WebDriver, Swagger, Gherkin]","[Java, Webdriver, Selenium, Swagger, BDD, Gher...","[Testes Funcionais, Testes Automatizados, Test...","[Aprendizado Contínuo, Proativo, Curiosidade, ...",0-2 anos,Júnior,true,Superior Completo,TI
6,4534,2021-2605711-MICROFOCUS QTP - UFT AUTOMATION T...,,26361,DOCUMENTAÇÃO PJ,2021-04-28,2021-05-11,AGUARDANDO CONFIRMAÇÃO DE INICIO _,MANUELLA CARVALHO,,...,TI,"[Selenium, Java, Eclipse, SQL, Azure DevOps, W...","[C, C#, WPF, Java, Selenio, Eclipse, SQL, Azur...","[Desenvolvimento, Testes, Automação, Regressão...","[Análise, Resolução, Implementação, Liderança,...",5-8 anos,Pleno,true,Superior Completo,Engenharia de Software
7,4534,2021-2605711-MICROFOCUS QTP - UFT AUTOMATION T...,,26205,DESISTIU,2021-04-23,2021-04-30,"PROFISSIONAL NÃO ATENDE OU RESPONDE LIGAÇÕES, ...",ANA CAMARGO,,...,TI,"[Selenium, Java, JavaScript, BDD, API, Testes,...","[Selenium Web Driver, WebDriver IO, Chai, Moch...","[Automação de testes, Desenvolvimento BDD, Tes...","[Proatividade, Análise Crítica, Resolução de P...",5-8 anos,Pleno,true,Superior Completo,TI
8,4534,2021-2605711-MICROFOCUS QTP - UFT AUTOMATION T...,,26003,NÃO APROVADO PELO CLIENTE,2021-04-08,2021-04-16,"CONVERSANDO COM A CANDIDATA, FOI EXPOSTO QUE A...",CAROLINA ARAÚJO,,...,TI,"[Selenium, Python,

In [5]:
# Combina as listas de skills
skills_extracted_df = resultado_final_df_polars.with_columns(
    (pl.col("job_ferramentas_tecnologicas").list.concat(pl.col("job_competencias_tecnicas"))).list.unique().alias("job_total_skills_list"),
    (pl.col("app_ferramentas_tecnologicas").list.concat(pl.col("app_competencias_tecnicas"))).list.unique().alias("app_total_skills_list")
)


In [6]:
def calculate_soft_match_score(job_skills, app_skills, model, threshold=0.5):
    """
    Calcula um score de similaridade semântica entre duas listas de skills.
    """
    # Retorna 0 se alguma das listas estiver vazia
    if not job_skills or not app_skills:
        return 0.0

    # Converte cada skill em um vetor numérico
    job_embeddings = model.encode(job_skills)
    app_embeddings = model.encode(app_skills)

    # Calcula a matriz de similaridade de cosseno entre todas as skills
    similarity_matrix = cosine_similarity(job_embeddings, app_embeddings)

    # Para cada skill da vaga, encontra a maior similaridade na lista do candidato
    best_matches = np.max(similarity_matrix, axis=1)

    # Aplica um threshold: se a melhor correspondência for muito baixa, vai considerar 0
    # Isso evita que "Excel" dê match com "Java"
    qualified_matches = [score if score >= threshold else 0 for score in best_matches]
    
    # O score final
    final_score = np.mean(qualified_matches)
    
    return final_score

In [7]:
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [8]:

skills_extracted_df = skills_extracted_df.with_columns(
    pl.struct(["job_total_skills_list", "app_total_skills_list"])
    .map_elements(
        lambda cols: calculate_soft_match_score(
            cols["job_total_skills_list"],
            cols["app_total_skills_list"],
            model
        ),
        return_dtype=pl.Float64
    ).alias("soft_match_score")
)

### FASE2

In [9]:

level_map = {
    # Idiomas
    'NENHUM': 0, 'BÁSICO': 1, 'INTERMEDIÁRIO': 2, 'AVANÇADO': 3, 'FLUENTE': 4, 'TÉCNICO': 1.5,
    
    # Níveis Profissionais
    'JÚNIOR': 1, 'JUNIOR': 1,
    'PLENO': 2,
    'SÊNIOR': 3, 'SENIOR': 3,
    'ESPECIALISTA': 4,
    'LÍDER': 5,
    
    # Níveis Acadêmicos
    'ENSINO MÉDIO COMPLETO': 1, 
    'ENSINO SUPERIOR INCOMPLETO': 2, 
    'ENSINO SUPERIOR COMPLETO': 3, 
    'PÓS-GRADUAÇÃO': 4,
    'MESTRADO': 4,
    'DOUTORADO': 4
}

#FAZ UM MAP DAS chaves do dicionário para upper e strip
def get_level(level_str):
    return level_map.get(str(level_str).upper().strip(), 0)

#
def calculate_structured_score(job_level, app_level):
    """
    Calcula uma pontuação proporcional baseada nos níveis.
    """
    job_score = get_level(job_level)
    app_score = get_level(app_level)

    # Se a vaga não exige nível, o candidato atende 100%
    if job_score == 0:
        return 1.0
    
    # Se o candidato não tem nível, a pontuação é 0
    if app_score == 0:
        return 0.0

    # Calcula a proporção e limita o score máximo a 1.0 (100%)
    ratio = app_score / job_score
    return min(ratio, 1.0)




In [10]:
skills_extracted_df = skills_extracted_df.with_columns(
    professional_level_score=pl.struct(["job_pv_nivel_profissional", "app_senioridade_aparente"]).map_elements(lambda x: calculate_structured_score(x['job_pv_nivel_profissional'], x['app_senioridade_aparente']), return_dtype=pl.Float64),
    academic_level_score=pl.struct(["job_pv_nivel_academico", "app_formacao_academica"]).map_elements(lambda x: calculate_structured_score(x['job_pv_nivel_academico'], x['app_formacao_academica']), return_dtype=pl.Float64),
    english_level_score=pl.struct(["job_pv_nivel_ingles", "app_fei_nivel_ingles"]).map_elements(lambda x: calculate_structured_score(x['job_pv_nivel_ingles'], x['app_fei_nivel_ingles']), return_dtype=pl.Float64)
)

In [11]:
structured_weights = {'professional': 0.5, 'academic': 0.3, 'english': 0.2}

skills_extracted_df = skills_extracted_df.with_columns(
    ((pl.col("professional_level_score") * structured_weights['professional']) +
     (pl.col("academic_level_score") * structured_weights['academic']) +
     (pl.col("english_level_score") * structured_weights['english']))
    .alias("structured_match_score")
)


### FASE 3

In [ ]:
job_text = skills_extracted_df["job_pv_principais_atividades"].fill_null("").to_list()
app_text = skills_extracted_df["app_cv_pt"].fill_null("").to_list()

job_embeddings = model.encode(job_text, show_progress_bar=False)
app_embeddings = model.encode(app_text, show_progress_bar=False)

cosine_scores = [cosine_similarity([job_embeddings[i]], [app_embeddings[i]])[0][0] for i in range(len(skills_extracted_df))]
skills_extracted_df = skills_extracted_df.with_columns(pl.Series("cosine_similarity_score", cosine_scores))

### FASE 4

In [ ]:
skills_extracted_df.group_by("codigo_vaga").agg([
    pl.col("cosine_similarity_score").max().alias("max_score_tecnico")
]).head()

codigo_vaga,max_score_tecnico
str,f32
"""4552""",0.298036
"""4744""",0.474382
"""4761""",0.559919
"""4545""",0.750607
"""4683""",0.610711


In [ ]:

#Calculando a pontuação final ponderada
final_weights = {
    'soft_match': 0.60,
    'cosine': 0.25,
    'structured': 0.15
}
skills_extracted_df = skills_extracted_df.with_columns(
    (pl.col("soft_match_score") * final_weights['soft_match'] +
     pl.col("structured_match_score") * final_weights['structured'] +
     pl.col("cosine_similarity_score") * final_weights['cosine'])
    .alias("technical_score")
)

technical_score_df = skills_extracted_df.select([
    "codigo_vaga",
    "codigo_candidato",
    pl.col("soft_match_score").round(3).alias("Score de Skills"),
    pl.col("structured_match_score").round(3).alias("Score Estruturado"),
    pl.col("cosine_similarity_score").round(3).alias("Score de Contexto"),
    pl.col("technical_score").round(3).alias("MATCH FINAL")
]).sort("MATCH FINAL", descending=True)

technical_score_df.sample(10).to_pandas()  # Exibe as primeiras 10 linhas do resultado final

,codigo_vaga,codigo_candidato,Score de Skills,Score Estruturado,Score de Contexto,MATCH FINAL
0,4660,20403,0.602,0.700,0.731,0.649
1,4601,24078,0.579,0.500,0.264,0.489
2,4647,28916,0.766,0.700,0.348,0.652
3,4589,9847,0.495,0.333,0.074,0.366
4,4741,25760,0.437,0.167,0.302,0.363
5,4660,23902,0.614,0.700,0.653,0.636
6,4724,25892,0.663,0.700,0.428,0.610
7,4615,25404,0.331,0.700,0.398,0.403
8,4696,15425,0.635,0.500,0.576,0.600
9,4633,25898,0.481,0.367,0.374,0.437


In [ ]:
technical_score_df.group_by("codigo_vaga").agg([
    pl.col("MATCH FINAL").max().alias("max_score_tecnico")
]).head()

codigo_vaga,max_score_tecnico
str,f64
"""4588""",0.572
"""4678""",0.447
"""4768""",0.561
"""4585""",0.719
"""4554""",0.598


In [ ]:
technical_score_df.sort("MATCH FINAL", descending=True)

codigo_vaga,codigo_candidato,Score de Skills,Score Estruturado,Score de Contexto,MATCH FINAL
str,str,f64,f64,f32,f64
"""7782""","""530""",0.809,1.0,0.866,0.852
"""4613""","""22165""",0.867,1.0,0.716,0.849
"""4613""","""16802""",0.84,1.0,0.675,0.823
"""4613""","""17318""",0.83,1.0,0.618,0.802
"""4680""","""6285""",0.877,0.7,0.68,0.801
…,…,…,…,…,…
"""4643""","""20314""",0.0,0.7,0.223,0.161
"""4643""","""22204""",0.0,0.633,0.182,0.14
"""4643""","""26819""",0.0,0.5,0.187,0.122


In [3]:
resultado_final_df_polars.head(10).to_dict()

{'codigo_vaga': shape: (10,)
 Series: 'codigo_vaga' [str]
 [
 	"4530"
 	"4530"
 	"4531"
 	"4531"
 	"4533"
 	"4533"
 	"4534"
 	"4534"
 	"4534"
 	"4534"
 ],
 'titulo': shape: (10,)
 Series: 'titulo' [str]
 [
 	"CONSULTOR CONTROL M"
 	"CONSULTOR CONTROL M"
 	"2021-2607395-PEOPLESOFT APPLIC…
 	"2021-2607395-PEOPLESOFT APPLIC…
 	"2021-2605708-MICROFOCUS APPLIC…
 	"2021-2605708-MICROFOCUS APPLIC…
 	"2021-2605711-MICROFOCUS QTP - …
 	"2021-2605711-MICROFOCUS QTP - …
 	"2021-2605711-MICROFOCUS QTP - …
 	"2021-2605711-MICROFOCUS QTP - …
 ],
 'modalidade': shape: (10,)
 Series: 'modalidade' [str]
 [
 	""
 	""
 	""
 	""
 	""
 	""
 	""
 	""
 	""
 	""
 ],
 'codigo_candidato': shape: (10,)
 Series: 'codigo_candidato' [str]
 [
 	"25632"
 	"25529"
 	"25364"
 	"25360"
 	"26338"
 	"24645"
 	"26361"
 	"26205"
 	"26003"
 	"25509"
 ],
 'p_situacao_candidado': shape: (10,)
 Series: 'p_situacao_candidado' [str]
 [
 	"ENCAMINHADO AO REQUISITANTE"
 	"ENCAMINHADO AO REQUISITANTE"
 	"CONTRATADO PELA DECISION"
 	